In [20]:
# Import all packages needed
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import json
from pathlib import Path
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [21]:
# Import style.json
style_path: Path = Path.cwd().parent / 'style' / 'style.json'
with open(style_path, 'r') as f:
    style = json.load(f)

In [22]:
# Loading Data (data file must be in the same directory as jupyter notebook)
# Import data from excel file
# TODO: replace filename with the name of your data file
raw_data_filename: str ='D-123-532-small range.dpt'

# Tell program to read the data
raw_data: pd.DataFrame = pd.read_csv(raw_data_filename, sep='\t', header=None)

# Output data
raw_data

,0,1
0,1520.0,39.14641
1,1520.5,39.14641
2,1521.0,39.14641
3,1521.5,39.14641
4,1522.0,39.14641
...,...,...
2416,2728.0,35.97698
2417,2728.5,35.97698
2418,2729.0,35.97698
2419,2729.5,35.97698


In [23]:
# Data preprocessing specific to RAMAN
def preprocess(data: pd.DataFrame, normalize: bool = False, baseline: bool = False):
    """Function that applies transformation to the dataframe which will make it ready for plotting. Note, this is specific to UV-Vis."""
    # Rename column names
    data = data.rename(
        columns={
            data.columns[0]: "Wavenumber ($\mathregular{cm^{-1}}$)",
            data.columns[1]: "Intensity (a.u.)",
        }
    )

    # Baseline correction
    if baseline:
        col: str = data.columns[1]
        data[col] = data[col] - np.average(data[col][0:50])

    # Normalize absorbance values
    if normalize:
        col: str = data.columns[1]
        max_val: float = data[col].max()
        data[col] = data[col] / max_val
        
    return data


# NOTE: change True to False if you don't want normalization or baseline correction
preprocessed_data: pd.DataFrame = preprocess(raw_data, False, False)
preprocessed_data

,Wavenumber ($\math{cm^{-1}}$),Intensity (a.u.)
0,1520.0,39.14641
1,1520.5,39.14641
2,1521.0,39.14641
3,1521.5,39.14641
4,1522.0,39.14641
...,...,...
2416,2728.0,35.97698
2417,2728.5,35.97698
2418,2729.0,35.97698
2419,2729.5,35.97698


In [24]:
# Plot the data!
# NOTE: User must change the sample names to the correct column names.
# seaborn styles
sns.set_style("ticks")
sns.set_context("notebook") #, rc={'figure.figsize':(11.7,8.27)})
# sns.set_theme(context="notebook", style="white", font="Helvetica")

graph = sns.lineplot(
    data=preprocessed_data,
    x="Wavenumber ($\mathregular{cm^{-1}}$)",
    y="Intensity (a.u.)",
    palette="rocket_r",  #NOTE: User can change the palette here
    linewidth=style["graph"]["line_thickness"],
)
sns.despine()

graph.legend(
    fontsize=style["legend"]["font_size"],
    bbox_to_anchor=(0.7, 0.7),
    title_fontsize=18,
    shadow=False,
    frameon=False,
)
# font family
plt.rcParams['font.family'] = style["graph"]["font_family"]

# y-axis and x-axis ticks
graph.xaxis.set_minor_locator(AutoMinorLocator(2))
graph.yaxis.set_minor_locator(AutoMinorLocator(2))
graph.tick_params(axis="y", direction="in")
graph.tick_params(axis="y", which="minor", length=4, direction="in")
graph.tick_params(axis="x", which="minor", length=4, direction="out")

# NOTE: User can change the x and y limits here
plt.xlim(1400, 2800)
# NOTE: User can change the y limits here
plt.ylim(-200, 1200)

plt.show()

/var/folders/ff/8h_dksfn11xfp_fq0srryncc0000gn/T/ipykernel_24756/3763431483.py:8: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  graph = sns.lineplot(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


ValueError: 
\math{cm^{-1}}
^
ParseFatalException: Unknown symbol: \math, found '\'  (at char 0), (line:1, col:1)

<Figure size 640x480 with 1 Axes>